#Data 

## 사진 400장 train/test set 나누기


In [49]:
def train_test(test_a,test_b):
  subject_train=[]
  for i in range(1,test_a):
    path = './att_faces/s'+str(i)
    file_list = os.listdir(path)
    for j in range(10):
      with Image.open(path+"/"+file_list[j]) as im:
        a= np.asarray(im)
        new_a=a.reshape(112*92,)
        subject_train.append(new_a)
  
  for i in range(test_b,41):
    path = './att_faces/s'+str(i)
    file_list = os.listdir(path)
    for j in range(10):
      with Image.open(path+"/"+file_list[j]) as im:
        a= np.asarray(im)
        new_a=a.reshape(112*92,)
        subject_train.append(new_a)
  subject_train=np.asarray(subject_train)

  subject_test=[]
  for i in range(test_a,test_b):
    path = './att_faces/s'+str(i)
    file_list = os.listdir(path)
    for j in range(10):
      with Image.open(path+"/"+file_list[j]) as im:
        a= np.asarray(im)
        new_a=a.reshape(112*92,)
        subject_test.append(new_a)
  subject_test=np.asarray(subject_test)
  
  return subject_train,subject_test

In [51]:
def Final_y_test(test_a,test_b,N_d):
  subject_test=train_test(test_a,test_b)[1]
  # 평균 빼서 데이터 원점으로 옮기기.
  subject_test_mean = np.mean(subject_test, axis=0, keepdims=True)
  subject_test_origin = subject_test - subject_test_mean

  #3차원으로 차원축소
  pca = my_PCA(N=N_d)
  pca.get_eigen(subject_test_origin)
  final_y_test=pca.dimension_reduction(subject_test_origin)[0]
  return final_y_test
def reconstruction(test_a,test_b):  
  subject_test=train_test(test_a,test_b)[1]
  N_eigen_matrics=result(test_a,test_b)[0]
  subject_test_mean = np.mean(subject_test, axis=0, keepdims=True)
  reconstructed_data = np.matmul(final_y_test,N_eigen_matrics.T)+subject_test_mean
  return reconstructed_data

In [53]:
def pointDist(x,y):
  sum=0
  for i in range(len(x)):
    sum+=(x[i]-y[i])**2
  dist=sum**(1/2)
  return dist


#LDA

##Fisher *face*

In [54]:
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt

In [56]:
def scatter(data):
  average_class=[]
  for i in range(len(data)):
      mean = np.mean(data[i], axis=0, keepdims=True)
      average_class.append(mean)
      
  average_class = np.asarray(average_class)

  s_w=0
  for s in range(len(data)):
      s1=0
      for i in range(len(data[s])):
        s1+=np.matmul((data[s][i]-average_class[s]).T,data[s][i]-average_class[s])
      s_w+=s1
  #s_w=s_w/40 #40
  mu= np.mean(average_class,axis=0)
  s_b=0
  for cc in range(len(data)):
    #s_b+=10*np.matmul((average_class[cc]-mu).T,average_class[cc]-mu)#10
    s_b+=np.matmul((average_class[cc]-mu).T,average_class[cc]-mu)  
  #s_b=s_b/40 #40
  return s_w,s_b

In [57]:
class my_lda():

    def __init__(self, N):
        #N(축소시킬 차원)
        self.N = N
        self.eigen_matrics = None
        self.eigen_value = None
        
    def get_eigen(self, data):
        data=data.reshape(len(data),10,10304)
        #data=data.reshape(2,5,2)
        s_ex_w=scatter(data)[0]
        s_ex_b=scatter(data)[1]
        s_w_inverse=np.linalg.pinv(s_ex_w)
        
        #data=data.reshape(len(data),10304)
        #data=data.reshape(10,2)        
        #n, d = data.shape # 400,10304
        d=10304
        eigen=np.matmul(s_w_inverse,s_ex_b)
        # np.linalg.eigh 이용해서
        #eigen value, eigen vector 계산하기
        eigvalues, eigvectors = np.linalg.eigh(eigen) 
        
        # eigen value와 vectors 짝지어서 저장.
        eigen_values_vectors = [(eigvalues[a], eigvectors[:, a]) for a in range(d)]
        
        # eigen value 값에 따라 내림차순 정렬 
        eigen_sorting = sorted(eigen_values_vectors, key=lambda tup: tup[0], reverse=True)
        
        # eigen vector 쌓아서 행렬 만들기. 
        self.eigen_matrics = np.stack(list(map(lambda tup: tup[1], eigen_sorting)), axis=1)
        
        # eigen value 모아서 벡터 만들기. 
        self.eigen_values = np.array(list(map(lambda tup: tup[0], eigen_sorting)))
        
        #sorting 안하고 순서대로 eigen matrics 뽑은거
        self.eigen_not_sorting_matrics = np.stack(list(map(lambda tup: tup[1], eigen_values_vectors)), axis=1)

    def dimension_reduction(self, data):
        
        # N개의 eigen vector로 행렬만들기.
        N_eigen_matrics = self.eigen_matrics[:, :self.N]
        
        # 차원 축소해서 정사영시킨 결과값들.
        final_y = np.matmul(data, N_eigen_matrics)

        return final_y,N_eigen_matrics, self.eigen_matrics, self.eigen_not_sorting_matrics

In [58]:
def result_lda(data,N_d):
  lda = my_lda(N=N_d)
  #eigen vector 구하기
  lda.get_eigen(data)

  #주성분 축 행렬로 만들기 and 확인
  N_eigen_matrics = lda.dimension_reduction(data)[1]
  #N_eigen_matrics.shape

  #차원축소 잘되었는지 확인
  final_y=lda.dimension_reduction(data)[0]
  #print(final_y.shape)
  eigen_matrics=lda.dimension_reduction(data)[2]

  #eigenface_vectors=N_eigen_matrics.T[0].reshape(112,92)
  #plt.imshow(eigenface_vectors)
  return N_eigen_matrics,final_y,eigen_matrics

##query and gallery(lda)

In [60]:
def ten_fold(sss,N_d):
    subject=train_test(sss,sss+4)
    subject_train=subject[0].reshape(36,10,10304)
    subject_test=subject[1].reshape(4,10,10304)
    
    results=result_lda(subject_train,N_d)
    N_eigen_matrics=results[0]
    final_y = np.matmul(subject_test, N_eigen_matrics)
    
    final_y_test=final_y.reshape(40,N_d)

    subject1=final_y_test[:10,:]
    subject2=final_y_test[10:20,:]
    subject3=final_y_test[20:30,:]
    subject4=final_y_test[30:41,:]

    subject_gallery=[]
    subject_gallery.append(subject1[3:11,:])
    subject_gallery.append(subject2[3:11,:])
    subject_gallery.append(subject3[3:11,:])
    subject_gallery.append(subject4[3:11,:])

    subject_gallery=np.asarray(subject_gallery).reshape(28,N_d)

    subject_query=[]
    subject_query.append(subject1[:3,:])
    subject_query.append(subject2[:3,:])
    subject_query.append(subject3[:3,:])
    subject_query.append(subject4[:3,:])
    subject_query=np.asarray(subject_query).reshape(12,N_d)

    dist=[]
    for abc in range(len(subject_query)):
          for gq in range(len(subject_gallery)):
                  dist.append(pointDist(subject_query[abc],subject_gallery[gq]))
    dist=np.asarray(dist)
    dist=dist.reshape(12,28)
    knn=np.argsort(dist, axis=1)[:, :7] 
    labeledKNN = knn//7

    c = []
    for row in labeledKNN:
      temp = [0 for i in range(4)]
      for i in range(7):
        temp[row[i]]+=1
      c.append(temp)
      
    splitedC = np.split(np.array(c), 4, axis=0)
    pred = []
    for row in splitedC:
      pred.append(np.sum(row, axis=0))
    pred=np.array(pred)/21.*100
    return pred

In [61]:
def TF(N_d):
  
  PPP=[]
  for i in range(1,38,4):
    PPP.append(ten_fold(i,N_d))
  PPP=np.array(PPP)
  Final=np.sum(PPP,axis=0)/10
  print("FINAL_CONFUSION_MATRICS",Final)

  accuracy=0
  for i in range(4):
    accuracy+=Final[i][i]
  accuracy=accuracy/4 
  print("accuracy",accuracy)
  return Final ,accuracy



###Dimension 3일때 Confusion matrix

In [62]:
r=TF(3)
r

FINAL_CONFUSION_MATRICS [[80.95238095 10.          0.          9.04761905]
 [ 8.57142857 81.9047619   4.28571429  5.23809524]
 [ 4.76190476 10.47619048 67.14285714 17.61904762]
 [11.42857143  5.71428571 18.57142857 64.28571429]]
accuracy 73.57142857142856


(array([[80.95238095, 10.        ,  0.        ,  9.04761905],
        [ 8.57142857, 81.9047619 ,  4.28571429,  5.23809524],
        [ 4.76190476, 10.47619048, 67.14285714, 17.61904762],
        [11.42857143,  5.71428571, 18.57142857, 64.28571429]]),
 73.57142857142856)